In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
from pylab import rcParams

import string
import math

In [2]:
# let's define a few functions we need
# gets an avg vector for an arbitrary count of vectors
def average_input(args):
    # get number of total inputs
    count = len(args)
    if(count == 0):
        return
    if(count == 1):
        return args[0][3:]
    avg = []
    # we know all input vectors will be of the same length
    # cut out vector values for index, animeID, and title_english
    for i in range(3, len(args[0])):
        isum = sum([vector[i] for vector in args])
        avg.append(isum/count)
    return avg

In [3]:
# returns cosine similarity between two vectors
def cosine_distance(vector1, vector2):
    # cosTHETA = (v1 dot v2)/(mag_v1 * mag_v2)
    # dot_prod = sum(tup[0] * tup[1] for tup in zip(vector1, vector2))
    dot_prod = np.dot(vector1, vector2)
    mag1 = np.sqrt(np.dot(vector1, vector1))
    mag2 = np.sqrt(np.dot(vector2, vector2))
    return (dot_prod / (mag1 * mag2))

In [4]:
# returns Euclidean distance between two vectors
def euclidean_distance(vector1, vector2):
    # d(x,y) = sqrt(sum((x[i]-y[i])^2 for all i in range(len(x))))
    sum = 0
    for i in range(len(vector1)):
        sum += (vector1[i]-vector2[i])**2
    return (sum**0.5)

In [5]:
# yeeeet time to implement K-Nearest Neighbors
# note: query is our average vector for the input anime data
def knn(query, data, dist, k):
    neighbois = []
    # time to yeet thru dem dataframe rows
    for index, row in data.iterrows():
        # get list for the current row
        # row includes animeID, title_english cols
        entry = list(row)[2:]
        # get the distance between query and current data point
        if(dist=="Euclidean"):
            distance = euclidean_distance(query, entry)
        else:
            distance = cosine_distance(query, entry)
        # append distance and index to list
        neighbois.append((distance, index))
    # time to sort list least to greatest distance or ascending (default)
    neighbois = sorted(neighbois)
    # nearest k-neighbors, by index label (animeID)
    k_nearest_neighbois = [boi[1] for boi in neighbois[:k]]
    # noice
    return k_nearest_neighbois

In [6]:
# time to clean out the input vectors from our data
# aka we can't recommend the animes user gave
def inputs_done_gone(big_boi_data, args):
    # get list of all indices of user input animes
    index_list = [anime[0] for anime in args]
    # remove from data
    clean_boi = big_boi_data.loc[~big_boi_data.index.isin(index_list)]
    return clean_boi

In [7]:
# method to get corresponding animes given anime title(s)??
def get_anime(data, titles):
    animes = []
    # iterate thru given titles
    for title in titles:
        # get all possible entries by title
        poss_anime = data[[title in item for item in data['title_english']]]
        if(len(poss_anime.index) == 0): # no possible animes
            print("Anime \'%s\' not found. Proceeding without \'%s\'."%(title,title))
        elif(len(poss_anime.index) == 1): # only one possible anime
            anime_data = [poss_anime.index] + list(poss_anime.loc[poss_anime.index[0]])
            full_title = anime_data[2] # index 2 is title
            print("Selected anime \'%s\'."%full_title)
            # add anime to list of anime
            animes.append(anime_data)
        else: # 1 or more possible title
            print(poss_anime[["animeID","title_english"]])
            index = input("Please enter indexes from above separated with commas ',' or -1 if absent: \n Ex. enter\'4,2,0\' for indexes 4, 2, and 0  ")
#           while(sum([index == i for i in poss_anime.index]) == 0 and index != -1):
#               index = input("Please enter indexes from above separated with commas ',' or -1 if absent: \n Ex. enter\'4,2,0\' for indexes 4, 2, and 0  ")
            if(index == "-1"):
                print("Anime \'%s\' not found. Proceeding without \'%s\'."%(title,title))
            else:
                indices = index.split(",") # array of index values, as str
                if(len(indices) == 1):
                    index = int(indices[0])
                    anime_data = [index] + list(poss_anime.loc[index])
                    full_title = anime_data[2] # index 2 is title_english
                    print("Selected anime \'%s\'."%full_title)
                    # add anime to list of anime
                    animes.append(anime_data)
                else: # more than one index found
                    for i in indices:
                        index = int(i)
                        anime_data = [index] + list(poss_anime.loc[index])
                        full_title = anime_data[2] # index 2 is title_english
                        print("Selected anime \'%s\'."%full_title)
                        # add anime to list of anime
                        animes.append(anime_data)
    return animes

In [8]:
############################################################################################################################
# LET'S RUN A FEW EXAMPLES USING OUR MODIFIED KNN ALGORITHM

In [9]:
# get output data (general info about animes) as well
output_df = pd.read_csv("data/relevant_output_data.csv")
# drop Unnamed: 0 column
output_df = output_df.drop(columns=["Unnamed: 0"])
output_df.head()

,animeID,title_english,synopsis
0,1,Cowboy Bebop,"In the year 2071, humanity has colonized sever..."
1,5,Cowboy Bebop: The Movie,"Another day, another bounty—such is the life o..."
2,6,Trigun,"Vash the Stampede is the man with a $$60,000,0..."
3,7,Witch Hunter Robin,Witches are individuals with special powers li...
4,8,Beet the Vandel Buster,It is the dark century and the people are suff...


In [10]:
# using model_training_data
anime_df = pd.read_csv("data/model_training_data.csv")
# remove Unnamed: 0 column
anime_df = anime_df.drop(columns=['Unnamed: 0'])
anime_df.head()

,animeID,title_english,episodes,score,scored_by,rank,popularity,members,favorites,genre_Action,...,synopsis_embedded_296,synopsis_embedded_297,synopsis_embedded_298,synopsis_embedded_299,synopsis_embedded_300,premiered_Spring,premiered_Summer,premiered_Fall,premiered_Winter,premiered_Year
0,1,Cowboy Bebop,26.0,8.81,405664,26,39,795733,43460,1,...,-0.015616,0.003619,-0.010637,0.043498,-0.007210,1.0,0.0,0.0,0.0,1998
1,5,Cowboy Bebop: The Movie,1.0,8.41,120243,164,449,197791,776,1,...,0.013802,0.005209,-0.006860,0.036505,-0.017165,0.0,0.0,0.0,0.0,0
2,6,Trigun,26.0,8.30,212537,255,146,408548,10432,1,...,-0.004276,0.010292,-0.036921,0.017824,0.005914,1.0,0.0,0.0,0.0,1998
3,7,Witch Hunter Robin,26.0,7.33,32837,2371,1171,79397,537,1,...,-0.008391,0.026849,-0.003479,0.048176,-0.002020,0.0,1.0,0.0,0.0,2002
4,8,Beet the Vandel Buster,52.0,7.03,4894,3544,3704,11708,14,0,...,-0.009359,-0.016920,-0.021534,0.047166,-0.012552,0.0,0.0,1.0,0.0,2004


In [11]:
# EXAMPLE 1 not-normalized, Cosine distance
# From a single anime title: ['Attack on Titan']
input_titles = ['Attack on Titan']
# get relevant data about our anime
my_anime_list = get_anime(anime_df, input_titles)
# print all titles from my_anime_list
print([anime[2] for anime in my_anime_list])

      animeID                           title_english
1810    16498                         Attack on Titan
1916    19285         Attack on Titan: Since That Day
2072    23775  Attack on Titan: Crimson Bow and Arrow
2073    23777       Attack on Titan: Wings of Freedom
2119    25777                Attack on Titan Season 2
2304    31374            Attack on Titan: Junior High
2705    35760                Attack on Titan Season 3
Please enter indexes from above separated with commas ',' or -1 if absent: 
 Ex. enter'4,2,0' for indexes 4, 2, and 0  1810
Selected anime 'Attack on Titan'.
['Attack on Titan']


In [12]:
# need to clean our inputs from overall database
clean_anime_df = inputs_done_gone(anime_df, my_anime_list)
clean_anime_df.head()

,animeID,title_english,episodes,score,scored_by,rank,popularity,members,favorites,genre_Action,...,synopsis_embedded_296,synopsis_embedded_297,synopsis_embedded_298,synopsis_embedded_299,synopsis_embedded_300,premiered_Spring,premiered_Summer,premiered_Fall,premiered_Winter,premiered_Year
0,1,Cowboy Bebop,26.0,8.81,405664,26,39,795733,43460,1,...,-0.015616,0.003619,-0.010637,0.043498,-0.007210,1.0,0.0,0.0,0.0,1998
1,5,Cowboy Bebop: The Movie,1.0,8.41,120243,164,449,197791,776,1,...,0.013802,0.005209,-0.006860,0.036505,-0.017165,0.0,0.0,0.0,0.0,0
2,6,Trigun,26.0,8.30,212537,255,146,408548,10432,1,...,-0.004276,0.010292,-0.036921,0.017824,0.005914,1.0,0.0,0.0,0.0,1998
3,7,Witch Hunter Robin,26.0,7.33,32837,2371,1171,79397,537,1,...,-0.008391,0.026849,-0.003479,0.048176,-0.002020,0.0,1.0,0.0,0.0,2002
4,8,Beet the Vandel Buster,52.0,7.03,4894,3544,3704,11708,14,0,...,-0.009359,-0.016920,-0.021534,0.047166,-0.012552,0.0,0.0,1.0,0.0,2004


In [13]:
# now, before we can run KNN, we need to get the average vector of our inputs
avg = average_input(my_anime_list)
print(avg)

[25.0, 8.48, 1038161, 116, 2, 1500958, 70555, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0.032716673247668214, 0.06146797841908981, 0.034833985932019286, 0.038697690379862884, -0.04890979066187022, 0.009864067544742507, 0.02381375371193399, -0.08152560798489318, 0.06261000341298628, 0.06667116710117885, -0.0057741865819814255, -0.07290000331645109, -0.02373649636093451, 0.04055233390963808, -0.07221115851888851, 0.03082856353448362, 0.025167971241230862, 0.07056349150988521, -0.005738472451969069, 0.000686781747000558, -0.01327912661494041, 0.015446487738161669, -0.020003493951291453, -0.006239618573869977, 0.04095747032944037, -0.026089522303367148, -0.05308373120366311, 0.052341485510067065, 0.024018735301737884, -0.020282511808434

In [14]:
# run KNN on our inputs: my_anime_list and k = 5 (for example)
rec_indices = knn(query=avg, data=clean_anime_df, dist="Cosine", k=5)
print(rec_indices)

[2743, 2774, 2738, 2850, 2732]


In [15]:
# Now, let's see what we were recommended!!
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: Gun Girls
SYNOPSIS: Qiang Niang mainly tells of the protagonist Carrie, who has lost her combat memories, searching for herself and the story of her final "Awakening". In the years between 2025 to 2030, a world war involving the entire world has resulted in the demise of half the world's population, and humanity has already lost control of the direction the war shall go in. In order to seize ultimate victory, war has gradually been turned into a battle without people, similar to a game. War has already become about Satellite lasers, Bomb drones, Intelligent AI, a battlefield between unmanned gun turrets and tanks... Then, there are also a group of young maidens who uses outdated firearms to engage in battle —— the Gun Girls.
TITLE: Dinosaurs Under Auroras
SYNOPSIS: A planetarium program that features dinosaurs living in the cold Alaskan climate.
TITLE: Nandaku-mou
SYNOPSIS: A music video for Kidori Kidori's song "Nandaku Mou."
TITLE: BUCK-TICK x HAL
SYNOPSIS: A music video colla

In [16]:
# using normalized data and 300 PCA components
norm_anime_df = pd.read_csv("data/normalized_princ_model_training_data.csv")
# remove Unnamed: 0 column
norm_anime_df = norm_anime_df.drop(columns=['Unnamed: 0'])
norm_anime_df.head()

,animeID,title_english,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,...,principal component 291,principal component 292,principal component 293,principal component 294,principal component 295,principal component 296,principal component 297,principal component 298,principal component 299,principal component 300
0,1,Cowboy Bebop,1.311131,-1.299874,-4.241516,2.813546,3.393908,0.748050,0.997277,-2.663760,...,0.370829,0.259884,-1.085205,-0.380120,-0.513250,0.240995,-0.466804,0.379151,-0.290872,-0.173427
1,5,Cowboy Bebop: The Movie,-0.877062,-0.658764,-6.071744,-2.503581,0.472272,-1.036241,-1.964439,0.466380,...,0.479847,0.018660,0.334813,0.040371,0.091011,0.700828,0.340917,0.102872,-0.480773,0.488478
2,6,Trigun,-2.958800,0.186702,-1.310995,-1.056926,3.076349,2.560385,-0.645996,-0.407952,...,0.097136,0.209499,-0.240368,-0.797081,-0.105798,-0.013730,0.300802,-0.055284,-0.164220,-0.205732
3,7,Witch Hunter Robin,0.579969,-0.164921,-3.543831,0.887435,-0.469471,1.760493,0.305682,-1.534360,...,-0.157432,-0.139492,-0.372866,0.935684,-0.077048,-0.116114,-0.245878,0.618540,-0.916615,-0.161398
4,8,Beet the Vandel Buster,-1.146832,1.695643,-5.688726,-0.873938,0.499840,-0.649930,0.549448,0.128640,...,0.271601,0.173991,-0.644193,0.232110,-0.148602,0.334401,-0.522669,0.118988,-0.303244,0.841451


In [17]:
# EXAMPLE 1, normalized with 300 components, Cosine distance
# From a single anime title: ['Attack on Titan']
# get relevant data about our anime
my_norm_anime_list = get_anime(norm_anime_df, input_titles)
# print all titles from my_anime_list
print([anime[2] for anime in my_norm_anime_list])

      animeID                           title_english
1810    16498                         Attack on Titan
1916    19285         Attack on Titan: Since That Day
2072    23775  Attack on Titan: Crimson Bow and Arrow
2073    23777       Attack on Titan: Wings of Freedom
2119    25777                Attack on Titan Season 2
2304    31374            Attack on Titan: Junior High
2705    35760                Attack on Titan Season 3
Please enter indexes from above separated with commas ',' or -1 if absent: 
 Ex. enter'4,2,0' for indexes 4, 2, and 0  1810
Selected anime 'Attack on Titan'.
['Attack on Titan']


In [18]:
# need to clean our inputs from normalized database
clean_norm_anime_df = inputs_done_gone(norm_anime_df, my_norm_anime_list)
clean_norm_anime_df.head()

,animeID,title_english,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,...,principal component 291,principal component 292,principal component 293,principal component 294,principal component 295,principal component 296,principal component 297,principal component 298,principal component 299,principal component 300
0,1,Cowboy Bebop,1.311131,-1.299874,-4.241516,2.813546,3.393908,0.748050,0.997277,-2.663760,...,0.370829,0.259884,-1.085205,-0.380120,-0.513250,0.240995,-0.466804,0.379151,-0.290872,-0.173427
1,5,Cowboy Bebop: The Movie,-0.877062,-0.658764,-6.071744,-2.503581,0.472272,-1.036241,-1.964439,0.466380,...,0.479847,0.018660,0.334813,0.040371,0.091011,0.700828,0.340917,0.102872,-0.480773,0.488478
2,6,Trigun,-2.958800,0.186702,-1.310995,-1.056926,3.076349,2.560385,-0.645996,-0.407952,...,0.097136,0.209499,-0.240368,-0.797081,-0.105798,-0.013730,0.300802,-0.055284,-0.164220,-0.205732
3,7,Witch Hunter Robin,0.579969,-0.164921,-3.543831,0.887435,-0.469471,1.760493,0.305682,-1.534360,...,-0.157432,-0.139492,-0.372866,0.935684,-0.077048,-0.116114,-0.245878,0.618540,-0.916615,-0.161398
4,8,Beet the Vandel Buster,-1.146832,1.695643,-5.688726,-0.873938,0.499840,-0.649930,0.549448,0.128640,...,0.271601,0.173991,-0.644193,0.232110,-0.148602,0.334401,-0.522669,0.118988,-0.303244,0.841451


In [19]:
# now, before we can run KNN, we need to get the average vector of our inputs
norm_avg = average_input(my_norm_anime_list)
print(norm_avg)

[-3.9251733362043666, -0.8041347483370096, -5.5623882983996085, 0.8110395349756183, 4.520072598937092, 5.113624753545662, -4.3192040668193705, -3.485466478489911, -1.6756276158269556, -0.7077401210900308, -7.904183810420007, -1.0332857527066897, -0.5420518367389274, -2.153947836215888, -1.1392744581148908, 2.04429783896998, -0.26157081721937503, 2.2187082865629706, -5.004791862479135, -0.9495444152877452, 4.0606954783542415, 2.2371445465711037, -0.7204816428466857, -1.3879002402931788, -0.12130187936589175, -0.4092753231293427, 3.152928272457124, -5.3303966704816155, 3.750182669842185, -2.514902174574132, -3.90491575331256, -2.4563743902678064, -0.3039789560580741, 3.4543909083152746, -1.105112970526818, -2.9037680503242105, -1.4274128423633163, -1.7299034544432177, -0.09138812580439133, -0.4387118217574753, -1.273562197129022, -2.593453856475288, 1.6647164218097934, 2.7383963552877844, -1.8806975812417377, -2.9278022833035418, 0.7327798772939014, -0.1239428992778952, -0.26115195361222

In [20]:
# run KNN on our inputs: my_anime_list and k = 5 (for example)
rec_indices = knn(query=norm_avg, data=clean_norm_anime_df, dist="Cosine", k=5)
print(rec_indices)

[494, 2481, 2230, 680, 2839]


In [21]:
# Now, let's see what we were recommended!!
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: Handsome Girl
SYNOPSIS: Hagiwara Mio is a 14-year-old TV actress who has gotten popular. One day on a set, she meets Kumagai Ichiya, who tells her that her acting "stinks." Mio is hurt by his comments, but she finds that she can't get Ichiya out of her mind. It turns out that Ichiya is a promising director, who directed a music video for Mio's friend Sawaki Aya, an idol singer. After the success of this music video, Ichiya is asked to direct a movie, and he wants Mio to be the heroine, because he feels that she is a "handsome" girl like the actresses of old. How will Mio cope with her feelings toward Ichiya? (Source: T.H.E.M. Anime Review, edited)
TITLE: Rainy Cocoa in Hawaii
SYNOPSIS: Ame-iro Cocoa: Rainy Cocoa Goes to Hawaii!! Nozomu Tokura, who is Aoi's older brother, is appointed as the manager of the Hawaii store. With more new characters, will he be able to open up the store there successfully?
TITLE: THE IDOLM@STER CINDERELLA GIRLS Special Program
SYNOPSIS: The iDOLM@STER

In [22]:
# EXAMPLE 1 not-normalized dataset, Euclidean distance measurement
# From a single anime title: ['Attack on Titan']
# already have relevant data about our anime in my_anime_list

# print all titles from my_anime_list
print([anime[2] for anime in my_anime_list])

['Attack on Titan']


In [23]:
# have clean_anime_df for dataset without our input
# also have avg for our average input vector
# run KNN on our inputs: my_anime_list and k = 5 (for example)
rec_indices = knn(query=avg, data=clean_anime_df, dist="Euclidean", k=5)
print(rec_indices)

[1612, 614, 1177, 2234, 2015]


In [24]:
# recommendations
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: Sword Art Online
SYNOPSIS: In the year 2022, virtual reality has progressed by leaps and bounds, and a massive online role-playing game called Sword Art Online (SAO) is launched. With the aid of "NerveGear" technology, players can control their avatars within the game using nothing but their own thoughts. Kazuto Kirigaya, nicknamed "Kirito," is among the lucky few enthusiasts who get their hands on the first shipment of the game. He logs in to find himself, with ten-thousand others, in the scenic and elaborate world of Aincrad, one full of fantastic medieval weapons and gruesome monsters. However, in a cruel turn of events, the players soon realize they cannot log out; the game's creator has trapped them in his new world until they complete all one hundred levels of the game. In order to escape Aincrad, Kirito will now have to interact and cooperate with his fellow players. Some are allies, while others are foes, like Asuna Yuuki, who commands the leading group attempting to esc

In [25]:
# EXAMPLE 1 normalized dataset with 300 PCA components, Euclidean distance measurement
# From a single anime title: ['Attack on Titan']
# already have relevant data about our anime in my_norm_anime_list

# print all titles from my_anime_list
print([anime[2] for anime in my_norm_anime_list])

['Attack on Titan']


In [26]:
# have clean_anime_df for dataset without our input
# also have norm_avg for our average input vector
# run KNN on our inputs: my_norm_anime_list and k = 5 (for example)
rec_indices = knn(query=norm_avg, data=clean_norm_anime_df, dist="Euclidean", k=5)
print(rec_indices)

[2119, 634, 912, 1177, 2010]


In [27]:
# recommendations
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: Attack on Titan Season 2
SYNOPSIS: For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and destroyed the outermost wall, Wall Maria. Forced to retreat behind Wall Rose, humanity waited with bated breath for the Titans to reappear and destroy their safe haven once more. In Shingeki no Kyojin Season 2, Eren Yeager and others of the 104th Training Corps have just begun to become full members of the Survey Corps. As they ready themselves to face the Titans once again, their preparations are interrupted by the invasion of Wall Rose—but all is not as it seems as more mysteries are unraveled. As the Survey Corps races to save the wall, they uncover more about the invading Titans and the dark secrets of their own members. [Written by MAL Rewrite]
T

In [28]:
############################################################################################################################

In [29]:
# EXAMPLE 2 not-normalized, Cosine Distance
# From a single series of anime: ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow', 'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High', 'Attack on Titan Season 3']
input_titles = ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow', 
                   'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High', 
                   'Attack on Titan Season 3']
# get relevant data about our anime
my_anime_list = get_anime(anime_df, input_titles)
# print all titles from my_anime_list
print([anime[2] for anime in my_anime_list])

      animeID                           title_english
1810    16498                         Attack on Titan
1916    19285         Attack on Titan: Since That Day
2072    23775  Attack on Titan: Crimson Bow and Arrow
2073    23777       Attack on Titan: Wings of Freedom
2119    25777                Attack on Titan Season 2
2304    31374            Attack on Titan: Junior High
2705    35760                Attack on Titan Season 3
Please enter indexes from above separated with commas ',' or -1 if absent: 
 Ex. enter'4,2,0' for indexes 4, 2, and 0  1810
Selected anime 'Attack on Titan'.
Selected anime 'Attack on Titan: Since That Day'.
Selected anime 'Attack on Titan: Crimson Bow and Arrow'.
Selected anime 'Attack on Titan: Wings of Freedom'.
Selected anime 'Attack on Titan Season 2'.
Selected anime 'Attack on Titan: Junior High'.
Selected anime 'Attack on Titan Season 3'.
['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow', 'Attack on Titan: Win

In [30]:
# need to clean our inputs from overall database
clean_anime_df = inputs_done_gone(anime_df, my_anime_list)
clean_anime_df.head()

,animeID,title_english,episodes,score,scored_by,rank,popularity,members,favorites,genre_Action,...,synopsis_embedded_296,synopsis_embedded_297,synopsis_embedded_298,synopsis_embedded_299,synopsis_embedded_300,premiered_Spring,premiered_Summer,premiered_Fall,premiered_Winter,premiered_Year
0,1,Cowboy Bebop,26.0,8.81,405664,26,39,795733,43460,1,...,-0.015616,0.003619,-0.010637,0.043498,-0.007210,1.0,0.0,0.0,0.0,1998
1,5,Cowboy Bebop: The Movie,1.0,8.41,120243,164,449,197791,776,1,...,0.013802,0.005209,-0.006860,0.036505,-0.017165,0.0,0.0,0.0,0.0,0
2,6,Trigun,26.0,8.30,212537,255,146,408548,10432,1,...,-0.004276,0.010292,-0.036921,0.017824,0.005914,1.0,0.0,0.0,0.0,1998
3,7,Witch Hunter Robin,26.0,7.33,32837,2371,1171,79397,537,1,...,-0.008391,0.026849,-0.003479,0.048176,-0.002020,0.0,1.0,0.0,0.0,2002
4,8,Beet the Vandel Buster,52.0,7.03,4894,3544,3704,11708,14,0,...,-0.009359,-0.016920,-0.021534,0.047166,-0.012552,0.0,0.0,1.0,0.0,2004


In [31]:
# now, before we can run KNN, we need to get the average vector of our inputs
avg = average_input(my_anime_list)
print(avg)

[9.142857142857142, 7.84857142857143, 249745.0, 1360.4285714285713, 846.1428571428571, 419428.4285714286, 11949.285714285714, 0.8571428571428571, 0.0, 0.14285714285714285, 0.0, 0.8571428571428571, 0.0, 0.8571428571428571, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.42857142857142855, 0.0, 0.42857142857142855, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7142857142857143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.7142857142857143, 0.8571428571428571, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.8571428571428571, 0.0, 0.2857142857142857, 0.0, 0.0, 0.0, 0.14285714285714285, 0.5714285714285714, 0.024153673956796424, 0.03497585827719707, 0.017160352141257332, 0.06206493335772593, 0.04137469930778564, 0.03424170176088753,

In [32]:
# run KNN on our inputs: my_anime_list and k = 5 (for example)
rec_indices = knn(query=avg, data=clean_anime_df, dist="Cosine", k=5)
print(rec_indices)

[2743, 2774, 2738, 2850, 2732]


In [33]:
# Get recommendations
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: Gun Girls
SYNOPSIS: Qiang Niang mainly tells of the protagonist Carrie, who has lost her combat memories, searching for herself and the story of her final "Awakening". In the years between 2025 to 2030, a world war involving the entire world has resulted in the demise of half the world's population, and humanity has already lost control of the direction the war shall go in. In order to seize ultimate victory, war has gradually been turned into a battle without people, similar to a game. War has already become about Satellite lasers, Bomb drones, Intelligent AI, a battlefield between unmanned gun turrets and tanks... Then, there are also a group of young maidens who uses outdated firearms to engage in battle —— the Gun Girls.
TITLE: Dinosaurs Under Auroras
SYNOPSIS: A planetarium program that features dinosaurs living in the cold Alaskan climate.
TITLE: Nandaku-mou
SYNOPSIS: A music video for Kidori Kidori's song "Nandaku Mou."
TITLE: BUCK-TICK x HAL
SYNOPSIS: A music video colla

In [34]:
# EXAMPLE 2 normalized with 300 PCA components, Cosine Distance
# From a single series of anime: ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow', 'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High', 'Attack on Titan Season 3']

# get relevant data about our anime
my_norm_anime_list = get_anime(norm_anime_df, input_titles)
# print all titles from my_anime_list
print([anime[2] for anime in my_norm_anime_list])

      animeID                           title_english
1810    16498                         Attack on Titan
1916    19285         Attack on Titan: Since That Day
2072    23775  Attack on Titan: Crimson Bow and Arrow
2073    23777       Attack on Titan: Wings of Freedom
2119    25777                Attack on Titan Season 2
2304    31374            Attack on Titan: Junior High
2705    35760                Attack on Titan Season 3
Please enter indexes from above separated with commas ',' or -1 if absent: 
 Ex. enter'4,2,0' for indexes 4, 2, and 0  1810
Selected anime 'Attack on Titan'.
Selected anime 'Attack on Titan: Since That Day'.
Selected anime 'Attack on Titan: Crimson Bow and Arrow'.
Selected anime 'Attack on Titan: Wings of Freedom'.
Selected anime 'Attack on Titan Season 2'.
Selected anime 'Attack on Titan: Junior High'.
Selected anime 'Attack on Titan Season 3'.
['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow', 'Attack on Titan: Win

In [35]:
# need to clean our inputs from overall database
clean_norm_anime_df = inputs_done_gone(norm_anime_df, my_norm_anime_list)
clean_norm_anime_df.head()

,animeID,title_english,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,...,principal component 291,principal component 292,principal component 293,principal component 294,principal component 295,principal component 296,principal component 297,principal component 298,principal component 299,principal component 300
0,1,Cowboy Bebop,1.311131,-1.299874,-4.241516,2.813546,3.393908,0.748050,0.997277,-2.663760,...,0.370829,0.259884,-1.085205,-0.380120,-0.513250,0.240995,-0.466804,0.379151,-0.290872,-0.173427
1,5,Cowboy Bebop: The Movie,-0.877062,-0.658764,-6.071744,-2.503581,0.472272,-1.036241,-1.964439,0.466380,...,0.479847,0.018660,0.334813,0.040371,0.091011,0.700828,0.340917,0.102872,-0.480773,0.488478
2,6,Trigun,-2.958800,0.186702,-1.310995,-1.056926,3.076349,2.560385,-0.645996,-0.407952,...,0.097136,0.209499,-0.240368,-0.797081,-0.105798,-0.013730,0.300802,-0.055284,-0.164220,-0.205732
3,7,Witch Hunter Robin,0.579969,-0.164921,-3.543831,0.887435,-0.469471,1.760493,0.305682,-1.534360,...,-0.157432,-0.139492,-0.372866,0.935684,-0.077048,-0.116114,-0.245878,0.618540,-0.916615,-0.161398
4,8,Beet the Vandel Buster,-1.146832,1.695643,-5.688726,-0.873938,0.499840,-0.649930,0.549448,0.128640,...,0.271601,0.173991,-0.644193,0.232110,-0.148602,0.334401,-0.522669,0.118988,-0.303244,0.841451


In [36]:
# now, before we can run KNN, we need to get the average vector of our inputs
norm_avg = average_input(my_norm_anime_list)
print(avg)

[9.142857142857142, 7.84857142857143, 249745.0, 1360.4285714285713, 846.1428571428571, 419428.4285714286, 11949.285714285714, 0.8571428571428571, 0.0, 0.14285714285714285, 0.0, 0.8571428571428571, 0.0, 0.8571428571428571, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.42857142857142855, 0.0, 0.42857142857142855, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7142857142857143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.0, 0.7142857142857143, 0.8571428571428571, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14285714285714285, 0.8571428571428571, 0.0, 0.2857142857142857, 0.0, 0.0, 0.0, 0.14285714285714285, 0.5714285714285714, 0.024153673956796424, 0.03497585827719707, 0.017160352141257332, 0.06206493335772593, 0.04137469930778564, 0.03424170176088753,

In [37]:
# run KNN on our inputs: my_anime_list and k = 5 (for example)
rec_indices = knn(query=norm_avg, data=clean_norm_anime_df, dist="Cosine", k=5)
print(rec_indices)

[2349, 2610, 2415, 329, 214]


In [38]:
# Get recommendations
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: Mahou Shoujo Nante Mouiidesukara
SYNOPSIS: Yuzuka Hanami is a young, carefree girl who lives the most ordinary life imaginable. Although her father works around the clock and her mother is rarely home, she still enjoys herself and strives to be an excellent student. Miton, on the other hand, is an alien life-form with the ability to transform his master into a magical girl, a warrior who fights evil wherever it may appear. However, there are not as many enemies as there used to be, so Miton has been out of work for a while. Starving and homeless, he has taken up residence in a pile of garbage. As Yuzuka walks past him one day, Miton seizes the opportunity to offer his services to the young girl. Yuzuka reluctantly agrees, but when she transforms into a magical girl and discovers that her outfit is a swimsuit, she begins to have second thoughts about what she has gotten herself into! [Written by MAL Rewrite]
TITLE: Chronos Ruler
SYNOPSIS: ​Like many in her class, Koyuki Honda loo

In [39]:
# EXAMPLE 2 not-normalized, Euclidean Distance
# From a single series of anime: ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow', 'Attack on Titan: Wings of Freedom',
#  'Attack on Titan Season 2', 'Attack on Titan: Junior High', 'Attack on Titan Season 3']

# have relevant data about our animes with avg and my_anime_list
print([anime[2] for anime in my_anime_list])

['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow', 'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High', 'Attack on Titan Season 3']


In [40]:
# run KNN on our inputs: my_anime_list and k = 5 (for example)
rec_indices = knn(query=avg, data=clean_anime_df, dist="Euclidean", k=5)
print(rec_indices)

[2477, 932, 2075, 2194, 1734]


In [41]:
# Get recommendations
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: Miss Kobayashi's Dragon Maid
SYNOPSIS: As Kobayashi sets off for another day at work, she opens her apartment door only to be met by an unusually frightening sight—the head of a dragon, staring at her from across the balcony. The dragon immediately transforms into a cute, busty, and energetic young girl dressed in a maid outfit, introducing herself as Tooru. It turns out that the stoic programmer had come across the dragon the previous night on a drunken excursion to the mountains, and since the mythical beast had nowhere else to go, she had offered the creature a place to stay in her home. Thus, Tooru had arrived to cash in on the offer, ready to repay her savior's kindness by working as her personal maidservant. Though deeply regretful of her words and hesitant to follow through on her promise, a mix of guilt and Tooru's incredible dragon abilities convinces Kobayashi to take the girl in. Despite being extremely efficient at her job, the maid's unorthodox methods of housekeepi

In [42]:
# EXAMPLE 2 normalized using 300 PCA components, Euclidean Distance
# From a single series of anime: ['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow', 'Attack on Titan: Wings of Freedom',
#  'Attack on Titan Season 2', 'Attack on Titan: Junior High', 'Attack on Titan Season 3']

# have relevant normalized data in nomr_avg and my_norm_anime_list
print([anime[2] for anime in my_norm_anime_list])

['Attack on Titan', 'Attack on Titan: Since That Day', 'Attack on Titan: Crimson Bow and Arrow', 'Attack on Titan: Wings of Freedom', 'Attack on Titan Season 2', 'Attack on Titan: Junior High', 'Attack on Titan Season 3']


In [43]:
# run KNN on our inputs: my_anime_list and k = 5 (for example)
rec_indices = knn(query=norm_avg, data=clean_norm_anime_df, dist="Euclidean", k=5)
print(rec_indices)

[1916, 2619, 1717, 2119, 427]


In [44]:
# Get recommendations
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: Attack on Titan: Since That Day
SYNOPSIS: Recap of the events in Shingeki no Kyojin that happened prior to episode 14.
TITLE: Marches Comes in Like a Lion: Special Omnibus Episode
SYNOPSIS: Recap of the first 11 episodes of 3-gatsu no Lion.
TITLE: Persona 4 the Animation: The Factor of Hope
SYNOPSIS: Recap of the entire 25-episode television anime series with new cuts and also the unaired "True End Episode" that was included in the 10th Blu-ray Disc/DVD volume.
TITLE: Attack on Titan Season 2
SYNOPSIS: For centuries, humanity has been hunted by giant, mysterious predators known as the Titans. Three mighty walls—Wall Maria, Rose, and Sheena—provided peace and protection for humanity for over a hundred years. That peace, however, was shattered when the Colossus Titan and Armored Titan appeared and destroyed the outermost wall, Wall Maria. Forced to retreat behind Wall Rose, humanity waited with bated breath for the Titans to reappear and destroy their safe haven once more. In Shin

In [45]:
############################################################################################################################

In [46]:
# EXAMPLE 3 not-normalized, Cosine distance
# From a relatively similar assortment of anime: ['Attack on Titan', 'Attack on Titan Season 2', 
#   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
#   'One Punch Man']
# let's get our input animes :D
input_titles = ['Attack on Titan', 'Attack on Titan Season 2', 'Bungo Stray Dogs',
                'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 
                'Nanbaka: Idiots with Student Numbers!','One Punch Man']

# print input titles
print(input_titles)

['Attack on Titan', 'Attack on Titan Season 2', 'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!', 'One Punch Man']


In [47]:
# time to get our anime data from overall training data
my_anime_list = get_anime(anime_df, input_titles)

# print all titles from my_anime_list
print([anime[2] for anime in my_anime_list])

      animeID                           title_english
1810    16498                         Attack on Titan
1916    19285         Attack on Titan: Since That Day
2072    23775  Attack on Titan: Crimson Bow and Arrow
2073    23777       Attack on Titan: Wings of Freedom
2119    25777                Attack on Titan Season 2
2304    31374            Attack on Titan: Junior High
2705    35760                Attack on Titan Season 3
Please enter indexes from above separated with commas ',' or -1 if absent: 
 Ex. enter'4,2,0' for indexes 4, 2, and 0  1810
Selected anime 'Attack on Titan'.
Selected anime 'Attack on Titan Season 2'.
      animeID       title_english
2316    31478    Bungo Stray Dogs
2424    32867  Bungo Stray Dogs 2
Please enter indexes from above separated with commas ',' or -1 if absent: 
 Ex. enter'4,2,0' for indexes 4, 2, and 0  2316
Selected anime 'Bungo Stray Dogs'.
Selected anime 'My Hero Academia 3'.
      animeID                          title_english
2222    30016   

In [48]:
# need to clean our inputs from overall database
clean_anime_df = inputs_done_gone(anime_df, my_anime_list)
clean_anime_df.head()

,animeID,title_english,episodes,score,scored_by,rank,popularity,members,favorites,genre_Action,...,synopsis_embedded_296,synopsis_embedded_297,synopsis_embedded_298,synopsis_embedded_299,synopsis_embedded_300,premiered_Spring,premiered_Summer,premiered_Fall,premiered_Winter,premiered_Year
0,1,Cowboy Bebop,26.0,8.81,405664,26,39,795733,43460,1,...,-0.015616,0.003619,-0.010637,0.043498,-0.007210,1.0,0.0,0.0,0.0,1998
1,5,Cowboy Bebop: The Movie,1.0,8.41,120243,164,449,197791,776,1,...,0.013802,0.005209,-0.006860,0.036505,-0.017165,0.0,0.0,0.0,0.0,0
2,6,Trigun,26.0,8.30,212537,255,146,408548,10432,1,...,-0.004276,0.010292,-0.036921,0.017824,0.005914,1.0,0.0,0.0,0.0,1998
3,7,Witch Hunter Robin,26.0,7.33,32837,2371,1171,79397,537,1,...,-0.008391,0.026849,-0.003479,0.048176,-0.002020,0.0,1.0,0.0,0.0,2002
4,8,Beet the Vandel Buster,52.0,7.03,4894,3544,3704,11708,14,0,...,-0.009359,-0.016920,-0.021534,0.047166,-0.012552,0.0,0.0,1.0,0.0,2004


In [49]:
# now, before we can run KNN, we need to get the average vector of our inputs
avg = average_input(my_anime_list)
print(avg)

[14.0, 8.0075, 353293.0, 1069.75, 727.625, 578835.625, 16320.625, 1.0, 0.0, 0.75, 0.0, 0.5, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.375, 0.0, 0.0, 0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.375, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125, 0.125, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.125, 0.0, 0.125, 0.25, 0.25, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125, 0.0, 0.0, 0.0, 0.0, 0.5, 0.5, 0.0, 0.0, 0.0, 0.125, 0.0, 0.125, 0.75, 0.029709092987177735, 0.04754042686001512, 0.026820564648707922, 0.03546425384641945, -0.029407804885756866, 0.001255215020571313, 0.01615228361871887, -0.0521324529199125, 0.05183349990396765, 0.0494942260655344, -0.009130671328129997, -0.0629351420264902, -0.0027844597786221515, 0.036917192202995476, -0.06918728142517463, 0.02350908639649838, 0.028550331983225614, 0.05850249797729284, -0.0032245527

In [50]:
# run KNN on our inputs: my_anime_list and k = 7 (for example)
rec_indices = knn(query=avg, data=clean_anime_df, dist="Cosine", k=5)
print(rec_indices)

[2743, 2774, 2738, 2850, 2732]


In [51]:
# Now, let's see what we were recommended!!
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: Gun Girls
SYNOPSIS: Qiang Niang mainly tells of the protagonist Carrie, who has lost her combat memories, searching for herself and the story of her final "Awakening". In the years between 2025 to 2030, a world war involving the entire world has resulted in the demise of half the world's population, and humanity has already lost control of the direction the war shall go in. In order to seize ultimate victory, war has gradually been turned into a battle without people, similar to a game. War has already become about Satellite lasers, Bomb drones, Intelligent AI, a battlefield between unmanned gun turrets and tanks... Then, there are also a group of young maidens who uses outdated firearms to engage in battle —— the Gun Girls.
TITLE: Dinosaurs Under Auroras
SYNOPSIS: A planetarium program that features dinosaurs living in the cold Alaskan climate.
TITLE: Nandaku-mou
SYNOPSIS: A music video for Kidori Kidori's song "Nandaku Mou."
TITLE: BUCK-TICK x HAL
SYNOPSIS: A music video colla

In [52]:
# EXAMPLE 3 normalized with 300 PCA components, Cosine distance
# From a relatively similar assortment of anime: ['Attack on Titan', 'Attack on Titan Season 2', 
#   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
#   'One Punch Man']
print(input_titles)

['Attack on Titan', 'Attack on Titan Season 2', 'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!', 'One Punch Man']


In [53]:
# time to get our anime data from normalized training data
my_norm_anime_list = get_anime(norm_anime_df, input_titles)

# print all titles from my_anime_list
print([anime[2] for anime in my_norm_anime_list])

      animeID                           title_english
1810    16498                         Attack on Titan
1916    19285         Attack on Titan: Since That Day
2072    23775  Attack on Titan: Crimson Bow and Arrow
2073    23777       Attack on Titan: Wings of Freedom
2119    25777                Attack on Titan Season 2
2304    31374            Attack on Titan: Junior High
2705    35760                Attack on Titan Season 3
Please enter indexes from above separated with commas ',' or -1 if absent: 
 Ex. enter'4,2,0' for indexes 4, 2, and 0  1810
Selected anime 'Attack on Titan'.
Selected anime 'Attack on Titan Season 2'.
      animeID       title_english
2316    31478    Bungo Stray Dogs
2424    32867  Bungo Stray Dogs 2
Please enter indexes from above separated with commas ',' or -1 if absent: 
 Ex. enter'4,2,0' for indexes 4, 2, and 0  2316
Selected anime 'Bungo Stray Dogs'.
Selected anime 'My Hero Academia 3'.
      animeID                          title_english
2222    30016   

In [54]:
# need to clean our inputs from overall database
clean_norm_anime_df = inputs_done_gone(norm_anime_df, my_norm_anime_list)
clean_norm_anime_df.head()

,animeID,title_english,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,...,principal component 291,principal component 292,principal component 293,principal component 294,principal component 295,principal component 296,principal component 297,principal component 298,principal component 299,principal component 300
0,1,Cowboy Bebop,1.311131,-1.299874,-4.241516,2.813546,3.393908,0.748050,0.997277,-2.663760,...,0.370829,0.259884,-1.085205,-0.380120,-0.513250,0.240995,-0.466804,0.379151,-0.290872,-0.173427
1,5,Cowboy Bebop: The Movie,-0.877062,-0.658764,-6.071744,-2.503581,0.472272,-1.036241,-1.964439,0.466380,...,0.479847,0.018660,0.334813,0.040371,0.091011,0.700828,0.340917,0.102872,-0.480773,0.488478
2,6,Trigun,-2.958800,0.186702,-1.310995,-1.056926,3.076349,2.560385,-0.645996,-0.407952,...,0.097136,0.209499,-0.240368,-0.797081,-0.105798,-0.013730,0.300802,-0.055284,-0.164220,-0.205732
3,7,Witch Hunter Robin,0.579969,-0.164921,-3.543831,0.887435,-0.469471,1.760493,0.305682,-1.534360,...,-0.157432,-0.139492,-0.372866,0.935684,-0.077048,-0.116114,-0.245878,0.618540,-0.916615,-0.161398
4,8,Beet the Vandel Buster,-1.146832,1.695643,-5.688726,-0.873938,0.499840,-0.649930,0.549448,0.128640,...,0.271601,0.173991,-0.644193,0.232110,-0.148602,0.334401,-0.522669,0.118988,-0.303244,0.841451


In [55]:
# need to get the average vector of our inputs
norm_avg = average_input(my_norm_anime_list)
print(norm_avg)

[-0.26187169826713597, -0.6223437198450654, -2.7640645151453818, 0.5428536450825734, -0.5229595898450449, 1.006645087521119, -2.301206722150935, -1.446522413144313, 0.2189986570852312, 0.1002835427889214, -2.801272981140133, -1.4157450024607294, -0.09573101635886633, -0.44875795393383616, 0.6688287518669537, 0.5816557632532964, 0.514554189552213, 1.525388903865321, -1.3692356232106808, -0.7028450294757473, 1.372415586368385, 0.7696922847505063, -0.19754721696318614, -0.7359311349023903, -0.13385817787648022, -0.1360728321921176, 0.48197149037105236, -0.5875646846501377, 1.050864796427776, -1.8522477987267276, -0.8279269225197536, -0.16700013821103804, 0.1981378795636637, 1.0252756506116198, -0.7430749812945876, -0.7998707221471164, 0.017967266879349936, -0.5023632492948615, 0.353978177484429, -0.5425766874173292, -0.4868964569313585, -0.320905686697994, -0.24655430083363838, 0.8888570453894349, -0.3600135371906611, -1.4550306337161716, 0.5090195753714379, -0.24458776770428842, -0.36730

In [56]:
# run KNN on our inputs: my_norm_anime_list and k = 5 (for example)
rec_indices = knn(query=norm_avg, data=clean_norm_anime_df, dist="Cosine", k=5)
print(rec_indices)

[1853, 2062, 704, 1399, 178]


In [57]:
# Get recommendations
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: Recently, my sister is unusual.
SYNOPSIS: Saikin, Imouto no Yousu ga Chotto Okashiinda ga. follows a family just starting to rebuild. When they marry, Mr. and Mrs. Kanzaki bring a teenage son and daughter along for the ride. But high school freshman Mitsuki Kanzaki is less than thrilled. Stinging from a history of absent and abusive father figures, she is slow to accept her stepfather and stepbrother. But after an accident lands Mitsuki in the hospital, she finds herself possessed by the ghost of Hiyori Kotobuki, a girl her age who was deeply in love with Mitsuki's stepbrother Yuuya. Hiyori cannot pass on to her final reward because of her unrequited love for Yuuya, meaning she's got to consummate it... in Mitsuki's body?! Now, Mitsuki's life depends on getting Hiyori to Heaven. But will she get used to sharing herself with a pushy, amorous ghost? Can she overcome her distrust of her new family? Can she bring herself to fulfill Hiyori's feelings for Yuuya? And might she be hidin

In [58]:
# EXAMPLE 3 not-normalized, Euclidean Distance
# From a relatively similar assortment of anime: ['Attack on Titan', 'Attack on Titan Season 2', 
#   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
#   'One Punch Man']
print([anime[2] for anime in my_anime_list])

['Attack on Titan', 'Attack on Titan Season 2', 'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!', 'One Punch Man']


In [59]:
# remember to use avg, and clean_anime_df
# run KNN on our inputs: my_anime_list and k = 5 (for example)
rec_indices = knn(query=avg, data=clean_anime_df, dist="Euclidean", k=5)
print(rec_indices)

[402, 1332, 1740, 96, 2213]


In [60]:
# Get recommendations
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: Ouran High School Host Club
SYNOPSIS: Haruhi Fujioka is a bright scholarship candidate with no rank or title to speak of—a rare species at Ouran High School, an elite academy for students of high pedigree. When she opens the door to Music Room #3 hoping to find a quiet place to study, Haruhi unexpectedly stumbles upon the Host Club. Led by the princely Tamaki, the club—whose other members include the "Shadow King" Kyouya, the mischievous Hitachiin twins, and the childlike Haninozuka "Honey" and his strong protector Mori—is where handsome boys with too much time on their hands entertain the girls in the academy. In a frantic attempt to remove herself from the hosts, Haruhi ends up breaking a vase worth eight million yen and is forced into becoming the eccentric group's general errand boy to repay her enormous debt. However, thanks to her convincingly masculine appearance, her naturally genial disposition toward girls, and fascinating commoner status, she is soon promoted to full-

In [61]:
# EXAMPLE 3 normalized, Euclidean Distance
# From a relatively similar assortment of anime: ['Attack on Titan', 'Attack on Titan Season 2', 
#   'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!',
#   'One Punch Man']
print([anime[2] for anime in my_norm_anime_list])

['Attack on Titan', 'Attack on Titan Season 2', 'Bungo Stray Dogs', 'My Hero Academia 3', 'Nanbaka', 'Nanbaka: Season 2', 'Nanbaka: Idiots with Student Numbers!', 'One Punch Man']


In [62]:
# remember to use norm_avg, and clean_norm_anime_df
# run KNN on our inputs: my_norm_anime_list and k = 5 (for example)
rec_indices = knn(query=norm_avg, data=clean_norm_anime_df, dist="Euclidean", k=5)
print(rec_indices)

[2362, 2609, 2010, 2302, 1960]


In [63]:
# Get recommendations
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: JoJo's Bizarre Adventure: Diamond is Unbreakable
SYNOPSIS: The year is 1999. Morioh, a normally quiet and peaceful town, has recently become a hotbed of strange activity. Joutarou Kuujou, now a marine biologist, heads to the mysterious town to meet Jousuke Higashikata. While the two may seem like strangers at first, Jousuke is actually the illegitimate child of Joutarou's grandfather, Joseph Joestar. When they meet, Joutarou realizes that he may have more in common with Jousuke than just a blood relation. Along with the mild-mannered Kouichi Hirose and the boisterous Okuyasu Nijimura, the group dedicates themselves to investigating recent disappearances and other suspicious occurrences within Morioh. Aided by the power of Stands, the four men will encounter danger at every street corner, as it is up to them to unravel the town's secrets, before another occurs. [Written by MAL Rewrite]
TITLE: Re:CREATORS
SYNOPSIS: Humans have designed countless worlds—each one born from the uniqu

In [64]:
############################################################################################################################

In [65]:
# EXAMPLE 4 not-normalized, Cosine distance
# a variety of genres (Horror, Ecchi, Comedy, Magic, Romance)
# From different animes: ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
input_titles = ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
# get relevant data about our anime
my_anime_list = get_anime(anime_df, input_titles)
# print all titles from my_anime_list
print([anime[2] for anime in my_anime_list])

Selected anime 'AKIRA'.
Selected anime 'Desert Punk'.
      animeID                                      title_english
7          20                                             Naruto
257       442  Naruto the Movie: Ninja Clash in the Land of Snow
438       936   Naruto the Movie 2: Legend of the Stone of Gelel
685      1735                                  Naruto: Shippuden
842      2472                        Naruto: Shippuden the Movie
1090     4134          Naruto Shippuden: Konoha Gakuen - Special
1115     4437              Naruto: Shippuden the Movie 2 -Bonds-
1553    10686    Chunin Exam on Fire! and Naruto vs. Konohamaru!
1705    13667                    Road to Ninja: Naruto the Movie
1823    16870                         The Last: Naruto the Movie
2182    28755                           Boruto: Naruto the Movie
Please enter indexes from above separated with commas ',' or -1 if absent: 
 Ex. enter'4,2,0' for indexes 4, 2, and 0  7
Selected anime 'Naruto'.
Selected anime 'D.N.

In [66]:
# need to clean our inputs from overall database
clean_anime_df = inputs_done_gone(anime_df, my_anime_list)
clean_anime_df.head()

,animeID,title_english,episodes,score,scored_by,rank,popularity,members,favorites,genre_Action,...,synopsis_embedded_296,synopsis_embedded_297,synopsis_embedded_298,synopsis_embedded_299,synopsis_embedded_300,premiered_Spring,premiered_Summer,premiered_Fall,premiered_Winter,premiered_Year
0,1,Cowboy Bebop,26.0,8.81,405664,26,39,795733,43460,1,...,-0.015616,0.003619,-0.010637,0.043498,-0.007210,1.0,0.0,0.0,0.0,1998
1,5,Cowboy Bebop: The Movie,1.0,8.41,120243,164,449,197791,776,1,...,0.013802,0.005209,-0.006860,0.036505,-0.017165,0.0,0.0,0.0,0.0,0
2,6,Trigun,26.0,8.30,212537,255,146,408548,10432,1,...,-0.004276,0.010292,-0.036921,0.017824,0.005914,1.0,0.0,0.0,0.0,1998
3,7,Witch Hunter Robin,26.0,7.33,32837,2371,1171,79397,537,1,...,-0.008391,0.026849,-0.003479,0.048176,-0.002020,0.0,1.0,0.0,0.0,2002
4,8,Beet the Vandel Buster,52.0,7.03,4894,3544,3704,11708,14,0,...,-0.009359,-0.016920,-0.021534,0.047166,-0.012552,0.0,0.0,1.0,0.0,2004


In [67]:
# now, before we can run KNN, we need to get the average vector of our inputs
avg = average_input(my_anime_list)
print(avg)

[73.0, 7.852000000000001, 237867.8, 1075.6, 430.8, 395422.0, 11273.2, 1.0, 0.8, 0.8, 0.0, 0.0, 0.2, 0.2, 0.0, 0.0, 0.2, 0.2, 0.0, 0.2, 0.2, 0.0, 0.0, 0.0, 0.4, 0.4, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.2, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.4, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.2, 0.0, 0.0, 0.0, 0.0, 0.6, 0.2, 0.2, 0.2, 0.0, 0.0, 0.0, 0.0, 0.8, 0.0429943793396914, 0.05832656809555007, 0.02909934861623966, 0.03818987578903877, -0.038576279730070606, 0.010655024739539696, 0.015538971883966765, -0.0555277989465897, 0.04745735150986852, 0.039205004534491786, -0.014447413663617046, -0.06733881026033307, -0.007012923370349705, 0.02912814107284113, -0.06538800971223893, 0.021779178982896368, 0.0318772138192239, 0.06415201469017003, 0.008624928021127855, -0.02050496556685626

In [68]:
# run KNN on our inputs: my_anime_list and k = 5 (for example)
rec_indices = knn(query=avg, data=clean_anime_df, dist="Cosine", k=5)
print(rec_indices)

[2743, 2774, 2738, 2850, 2732]


In [69]:
# Now, let's see what we were recommended!!
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: Gun Girls
SYNOPSIS: Qiang Niang mainly tells of the protagonist Carrie, who has lost her combat memories, searching for herself and the story of her final "Awakening". In the years between 2025 to 2030, a world war involving the entire world has resulted in the demise of half the world's population, and humanity has already lost control of the direction the war shall go in. In order to seize ultimate victory, war has gradually been turned into a battle without people, similar to a game. War has already become about Satellite lasers, Bomb drones, Intelligent AI, a battlefield between unmanned gun turrets and tanks... Then, there are also a group of young maidens who uses outdated firearms to engage in battle —— the Gun Girls.
TITLE: Dinosaurs Under Auroras
SYNOPSIS: A planetarium program that features dinosaurs living in the cold Alaskan climate.
TITLE: Nandaku-mou
SYNOPSIS: A music video for Kidori Kidori's song "Nandaku Mou."
TITLE: BUCK-TICK x HAL
SYNOPSIS: A music video colla

In [70]:
# EXAMPLE 4 normalize with 300 PCA components, Cosine distance
# a variety of genres (Horror, Ecchi, Comedy, Magic, Romance)
# From different animes: ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
# get relevant data about our anime
my_norm_anime_list = get_anime(norm_anime_df, input_titles)
# print all titles from my_anime_list
print([anime[2] for anime in my_norm_anime_list])

Selected anime 'AKIRA'.
Selected anime 'Desert Punk'.
      animeID                                      title_english
7          20                                             Naruto
257       442  Naruto the Movie: Ninja Clash in the Land of Snow
438       936   Naruto the Movie 2: Legend of the Stone of Gelel
685      1735                                  Naruto: Shippuden
842      2472                        Naruto: Shippuden the Movie
1090     4134          Naruto Shippuden: Konoha Gakuen - Special
1115     4437              Naruto: Shippuden the Movie 2 -Bonds-
1553    10686    Chunin Exam on Fire! and Naruto vs. Konohamaru!
1705    13667                    Road to Ninja: Naruto the Movie
1823    16870                         The Last: Naruto the Movie
2182    28755                           Boruto: Naruto the Movie
Please enter indexes from above separated with commas ',' or -1 if absent: 
 Ex. enter'4,2,0' for indexes 4, 2, and 0  7
Selected anime 'Naruto'.
Selected anime 'D.N.

In [71]:
# need to clean our inputs from normalized database
clean_norm_anime_df = inputs_done_gone(norm_anime_df, my_norm_anime_list)
clean_norm_anime_df.head()

,animeID,title_english,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5,principal component 6,principal component 7,principal component 8,...,principal component 291,principal component 292,principal component 293,principal component 294,principal component 295,principal component 296,principal component 297,principal component 298,principal component 299,principal component 300
0,1,Cowboy Bebop,1.311131,-1.299874,-4.241516,2.813546,3.393908,0.748050,0.997277,-2.663760,...,0.370829,0.259884,-1.085205,-0.380120,-0.513250,0.240995,-0.466804,0.379151,-0.290872,-0.173427
1,5,Cowboy Bebop: The Movie,-0.877062,-0.658764,-6.071744,-2.503581,0.472272,-1.036241,-1.964439,0.466380,...,0.479847,0.018660,0.334813,0.040371,0.091011,0.700828,0.340917,0.102872,-0.480773,0.488478
2,6,Trigun,-2.958800,0.186702,-1.310995,-1.056926,3.076349,2.560385,-0.645996,-0.407952,...,0.097136,0.209499,-0.240368,-0.797081,-0.105798,-0.013730,0.300802,-0.055284,-0.164220,-0.205732
3,7,Witch Hunter Robin,0.579969,-0.164921,-3.543831,0.887435,-0.469471,1.760493,0.305682,-1.534360,...,-0.157432,-0.139492,-0.372866,0.935684,-0.077048,-0.116114,-0.245878,0.618540,-0.916615,-0.161398
4,8,Beet the Vandel Buster,-1.146832,1.695643,-5.688726,-0.873938,0.499840,-0.649930,0.549448,0.128640,...,0.271601,0.173991,-0.644193,0.232110,-0.148602,0.334401,-0.522669,0.118988,-0.303244,0.841451


In [72]:
# get the average vector of our inputs
norm_avg = average_input(my_norm_anime_list)
print(norm_avg)

[-2.224826639558806, -0.14939572585115837, -0.3207085675662861, 1.5508571260618205, 1.8910910075846075, 1.603978375201877, -2.3220950488165095, -0.16483840479131523, 1.0090639262336718, -0.9640535196876222, -2.342128233513717, -2.2274351408947295, -0.4523796284711438, -1.1989433312486788, 1.116371779125099, -0.09521308961251974, 0.5171272317193187, 0.12783013734184684, -0.018668464911993966, -0.5777996864537627, 0.8295077652313136, 1.2485711147570022, 0.0020874986107689785, 0.2857881970590078, 0.6589374317233148, 0.5534666275969952, 1.0278025963720903, -0.7584755771443745, 0.16907952317601424, -0.7944899277140657, -0.6796745784028255, -0.8102773709343551, 0.2824891046698507, 0.14947912259756685, -0.5125056394270427, 0.8018935293711449, -0.3490268161296086, -0.6688157892016167, 0.19172788238425892, -0.20287482370207646, 0.9255461155366056, -0.12812004628785653, -0.26268127204293973, 0.32786240577716935, 0.2359424285225161, -0.3298479897582868, 0.5993755751175602, -0.3427222393685404, -0

In [73]:
# run KNN on our inputs: my_norm_anime_list and k = 5 (for example)
rec_indices = knn(query=norm_avg, data=clean_norm_anime_df, dist="Cosine", k=5)
print(rec_indices)

[1232, 2758, 2778, 1933, 2679]


In [74]:
# Get recommendations
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: Cinnamoroll: The Movie
SYNOPSIS: Mocha, Chiffon, Cappuccino, Espresso, and Milk go with Cinnamoroll in this adventure of a lifetime. Join them in their quest, meeting new friends like Anna (the girl who found Cinnamoroll/Cinnamon) and discovering new places like the Forest of Pastries and Bread and the Coffee Waterfalls.
TITLE: THE IDOLM@STER CINDERELLA GIRLS Theater (Web) 2nd Season
SYNOPSIS: Bonus segments of Cinderella Girls Gekijou 2nd Season streamed on The iDOLM@STER Cinderella Girls Mobage following the television broadcast.
TITLE: Working Buddies!
SYNOPSIS: The story centers around the cheerful calico Tapio Chatorazawa and the pessimistic Russian blue Kuehiko Roshihara. The two study in the same college, and also work together on various odd jobs. (Source: ANN)
TITLE: Robot Girls Z
SYNOPSIS: The story revolves around a new team of "Toei robot girls" named "Team Z" that is made up of three girls based on Toei's Mazinger Z franchise: Mazinger Z, Great Mazinger, and UFO Rob

In [75]:
# EXAMPLE 4 not-normalized, Euclidean distance
# a variety of genres (Horror, Ecchi, Comedy, Magic, Romance)
# From different animes: ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
# relevant data about our anime: my_anime_list, avg
print([anime[2] for anime in my_anime_list])

['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']


In [76]:
# run KNN on our inputs: my_anime_list and k = 5 (for example)
rec_indices = knn(query=avg, data=clean_anime_df, dist="Euclidean", k=5)
print(rec_indices)

[2193, 1600, 844, 1646, 1825]


In [77]:
# Get recommendations
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: Haikyu!! 2nd Season
SYNOPSIS: Following their participation at the Inter-High, the Karasuno High School volleyball team attempts to refocus their efforts, aiming to conquer the Spring tournament instead. When they receive an invitation from long-standing rival Nekoma High, Karasuno agrees to take part in a large training camp alongside many notable volleyball teams in Tokyo and even some national level players. By playing with some of the toughest teams in Japan, they hope not only to sharpen their skills, but also come up with new attacks that would strengthen them. Moreover, Hinata and Kageyama attempt to devise a more powerful weapon, one that could possibly break the sturdiest of blocks. Facing what may be their last chance at victory before the senior players graduate, the members of Karasuno's volleyball team must learn to settle their differences and train harder than ever if they hope to overcome formidable opponents old and new—including their archrival Aoba Jousai and 

In [78]:
# EXAMPLE 4 normalized, Euclidean distance
# a variety of genres (Horror, Ecchi, Comedy, Magic, Romance)
# From different animes: ['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']
# relevant data about our anime: my_norm_anime_list, avg
print([anime[2] for anime in my_norm_anime_list])

['AKIRA', 'Desert Punk', 'Naruto', 'D.N.Angel', 'Rurouni Kenshin']


In [79]:
# run KNN on our inputs: my_norm_anime_list and k = 5 (for example)
rec_indices = knn(query=norm_avg, data=clean_norm_anime_df, dist="Euclidean", k=5)
print(rec_indices)

[1960, 2302, 1734, 1045, 2609]


In [80]:
# Get recommendations
for i in rec_indices:
    details = list(output_df.loc[i])
    print("TITLE: %s"%details[1])
    print("SYNOPSIS: %s"%details[2])

TITLE: JoJo's Bizarre Adventure: Stardust Crusaders
SYNOPSIS: Years after an ancient evil was salvaged from the depths of the sea, Joutarou Kuujou sits peacefully within a Japanese jail cell. He's committed no crime yet demands he not be released, believing he's been possessed by an evil spirit capable of harming those around him. Concerned for her son, Holly Kuujou asks her father, Joseph Joestar, to convince Joutarou to leave the prison. Joseph informs his grandson that the "evil spirit" is in fact something called a "Stand," the physical manifestation of one's fighting spirit which can adopt a variety of deadly forms. After a fiery brawl with Joseph's friend Mohammed Avdol, Joutarou is forced out of his cell and begins learning how to control the power of his Stand. However, when a Stand awakens within Holly and threatens to consume her in 50 days, Joutarou, his grandfather, and their allies must seek out and destroy the immortal vampire responsible for her condition. They must trav